In [1]:
import requests
import zipfile
import io
import psycopg2
import csv
import os
from dotenv import load_dotenv

def lambda_handler():
    
    # Download the zip file
    url = "https://www.fracfocusdata.org/digitaldownload/FracFocusCSV.zip"
    response = requests.get(url)
    zip_bytes = io.BytesIO(response.content)
    load_dotenv(dotenv_path=".env")


    # Extract CSV from ZIP
    with zipfile.ZipFile(zip_bytes) as zip_file:
        csv_filename = [f for f in zip_file.namelist() if f.endswith('.csv')][0]
        with zip_file.open(csv_filename) as csvfile:
            decoded = io.TextIOWrapper(csvfile)
            reader = csv.reader(decoded)
            headers = next(reader)

            # Connect to PostgreSQL
            conn = psycopg2.connect(
                host=os.environ['DB_HOST'],
                database=os.environ['DB_NAME'],
                user=os.environ['DB_USER'],
                password=os.environ['DB_PASSWORD'],
                port=5432
            )
            cur = conn.cursor()

            # Drop and recreate table
            cur.execute(f"DROP TABLE IF EXISTS frac_data;")
            create_stmt = f"CREATE TABLE frac_data ({', '.join([f'{col} TEXT' for col in headers])});"
            cur.execute(create_stmt)

            # Copy data
            insert_stmt = f"INSERT INTO frac_data VALUES ({', '.join(['%s'] * len(headers))})"
            for row in reader:
                cur.execute(insert_stmt, row)

            conn.commit()
            cur.close()
            conn.close()

    return {"status": "success"}


In [2]:
lambda_handler()

KeyboardInterrupt: 